# EBAC - Profissão Cientista de Dados : Módulo 23 - Combinação de Modelos I - Exercício 1



**1) Monte um passo a passo para o Bagging**

**Passo 1 - Amostragem com reposição (Bootstrap)**
- Criamos um número x de conjuntos de dados a partir da base de dados original, através de uma amostragem com reposição, ou seja, cada elemento de cada conjunto de dados será escolhido aleatoriamente da base de dados original, tendo a possibilidade de ser selecionado novamente.

**Passo 2 - Criação dos Base Learners**

- Escolhemos um modelo adequado para o problema;
- Treinamos um modelo separado para cada conjunto de treinamento gerado na etapa anterior.

**Passo 3 - Agregação (Agreggating)**
- Combinamos as previsões de cada um do modelos para obter uma previsão final, por exemplo usando o resultado da maioria dos modelos para problemas de classificação ou calculando a média das previsões para problemas de regressão.


**2) Explique com suas palavras o Bagging**

O Bagging (Bootstrap Aggregating) é uma técnica de aprendizado que envolve a combinação de múltiplos modelos preditivos para melhorar a precisão  das previsões. Ele opera em três etapas principais: amostragem por repetição (bootstrap), criação dos modelos base (Base Learnes) e agregação das previsões (aggregating). 
A finalidade dessa técnica é reduzir o viés e a variância dos modelos individuais, resultando em um modelo mais robusto e com menor probabilidade de overfitting.

**3) (Opcional) Implementar em python o código do Bagging**

In [1]:
#importando as bibliotecas

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
#Criando Dataframe Treino

# x - variável explicativa
# Gerar 5 números aleatórios entre 1 e 10
numeros_aleatorios_x = np.random.randint(1, 11, size=10)
# y -  variável resposta
# Gerar 5 números aleatórios entre 0 e 1
numeros_aleatorios_y = np.random.randint(0, 2, size=10)

# Criar DataFrame
df_treino = pd.DataFrame({'x': numeros_aleatorios_x,
                  'y': numeros_aleatorios_y
                 })

#exibir Dataframe
df_treino

,x,y
0,4,1
1,8,1
2,2,0
3,5,0
4,8,0
5,3,0
6,7,0
7,5,0
8,1,1
9,8,0


In [3]:
#Criando Dataframe Teste

# x - variável explicativa
# Gerar 5 números aleatórios entre 1 e 10
numeros_aleatorios_x = np.random.randint(1, 11, size=3)
# y -  variável resposta
# Gerar 5 números aleatórios entre 0 e 1
numeros_aleatorios_y = np.random.randint(0, 2, size=3)

# Criar DataFrame
df_teste = pd.DataFrame({'x': numeros_aleatorios_x,
                  'y': numeros_aleatorios_y
                 })

#exibir Dataframe
df_teste

,x,y
0,5,0
1,5,0
2,9,1


In [4]:
# Criando funções para cada passo
# Passo 1: Amostragem por repetição (Bootstrap)
def bootstrap_sampling(dados, num_amostras):
    amostras = []
    for _ in range(num_amostras):
        amostra = dados.sample(frac=1, replace=True)
        amostra.reset_index(drop = True, inplace= True)
        amostras.append(amostra)
    return amostras


# Passo 2: Criação dos Modelos Base Learners
def criar_base_learners(amostras):
    base_learners = []
    for amostra in amostras:
        X = amostra.iloc[:, :-1]  # Seleciona as Características (x) do df -assumindo que a última coluna do df sempre será a coluna y
        y = amostra.iloc[:, -1]   # Seleciona os Rótulos (y) do df - assumindo que a última coluna do df sempre será a coluna y
        learner = DecisionTreeClassifier() # cria um modelo de árvore de classificação
        learner.fit(X, y) # treina o modelo
        base_learners.append(learner) #adiciona  o modelo a lista 
    return base_learners

# Passo 3: Agregação
def agregar_modelos(modelos, X):
    predicoes = [modelo.predict(X) for modelo in modelos] # Cria uma lista contendo um array para cado um dos modelo criados, cada array contendo um valor predito para cada linha de X_teste. Considera que o y é a última coluna do df
    df_predicoes = pd.DataFrame(predicoes).T #transforma os arrays em dataframe e transpõe as linhas em colunas de modo que cada coluna do array seja a predição de um dos modelos
    colunas = [ 'Y Predito Modelo '+ str(i) for i in range(1, df_predicoes.shape[1] +1 )] # Cria nomes para cada uma das colunas do dataframe
    df_predicoes.columns = colunas # renomeia as colunas com os nomes criados
    #o resultado final das predições será o valor/categoria que foi predito mais vezes pelos modelos (ou seja a moda das linhas do df_predicoes)
    df_resultados = df_predicoes.iloc[:,2:].mode(axis=1).rename(columns = {0: 'y_previsto_final'})
    
   
    return df_resultados

**Aplicando as funções**

In [5]:
# Passo 1: Amostragem por repetição (Bootstrap)
amostras = bootstrap_sampling(df_treino, 5)

In [6]:
#Exemplos de amostras geradas
amostras[0]

,x,y
0,5,0
1,8,0
2,4,1
3,8,0
4,3,0
5,8,0
6,8,0
7,5,0
8,8,1
9,5,0


In [7]:
amostras[4]

,x,y
0,7,0
1,5,0
2,8,0
3,7,0
4,5,0
5,5,0
6,4,1
7,5,0
8,1,1
9,8,0


In [8]:
# Passo 2: Criação dos Modelos - Base Learners
# Vão ser criados um modelo de árvore de decisão para cada um dos 5 conjuntos de dados que criamos retirando amostras de df_treino
modelos = criar_base_learners(amostras)
modelos

[DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier()]

In [9]:
# Passo 3: Agregação
X_teste = df_teste.iloc[:,:-1] #considerando que o y é a última coluna do dataframe, pegamos todos as demais colunas que são valores de x
agregar_modelos(modelos, X_teste)

,y_previsto_final
0,0
1,0
2,0
